In [1]:
import estimate, rospy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
%matplotlib qt

<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<double, std::allocator<double> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:219: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignored.
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-root'


In [2]:
rospy.init_node('test')
sampler = estimate.WorkspaceSampler(
    'base_footprint', '/kinematics_server/check_collision', '/kinematics_server/goal_pose', 
    (10, 10, 10), [-1.25, -0.45981], [-0.75, 0.75], [0.0, 1.5], prefix='ur_'
)

[INFO] [1721846497.769151]: Connected to kinematics action server


In [3]:
reached, rejected = sampler.sample_reachable_points()

In [4]:
reached = np.array(reached)
rejected = np.array(rejected)
rxs, rys, rzs = reached[:, 0, 3], reached[:, 1, 3], reached[:, 2, 3]
nxs, nys, nzs = rejected[:, 0, 3], rejected[:, 1, 3], rejected[:, 2, 3]
pos = np.stack((rxs, rys, rzs), axis=1)

In [5]:
def cuboid_data(o, size=(1,1,1)):
    X = [[[0, 1, 0], [0, 0, 0], [1, 0, 0], [1, 1, 0]],
         [[0, 0, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]],
         [[1, 0, 1], [1, 0, 0], [1, 1, 0], [1, 1, 1]],
         [[0, 0, 1], [0, 0, 0], [0, 1, 0], [0, 1, 1]],
         [[0, 1, 0], [0, 1, 1], [1, 1, 1], [1, 1, 0]],
         [[0, 1, 1], [0, 0, 1], [1, 0, 1], [1, 1, 1]]]
    X = np.array(X).astype(float)
    for i in range(3):
        X[:,:,i] *= size[i]
    X += np.array(o)
    return X

def plotCubeAt(position, size=None, color='C0', **kwargs):
    return Poly3DCollection(cuboid_data(position, size=size), **kwargs)

In [6]:
def isin_tolerance(A, B, tol):
    A = np.asarray(A)
    B = np.asarray(B)

    Bs = np.sort(B) # skip if already sorted
    idx = np.searchsorted(Bs, A)

    linvalid_mask = idx==len(B)
    idx[linvalid_mask] = len(B)-1
    lval = Bs[idx] - A
    lval[linvalid_mask] *=-1

    rinvalid_mask = idx==0
    idx1 = idx-1
    idx1[rinvalid_mask] = 0
    rval = A - Bs[idx1]
    rval[rinvalid_mask] *=-1
    return np.minimum(lval, rval) <= tol

In [57]:
class VoxelPlotter:
    def __init__(self, fig, ax, sampler, pos) -> None:
        
        try:
            matplotlib.rcParams['keymap.back'].remove('left')
            matplotlib.rcParams['keymap.forward'].remove('right')
        except: pass
        
        self.index = -1
        self.fig = fig
        self.fig.canvas.mpl_connect('key_press_event', self.handle_keypress)
        self.ax = ax
        
        self.sampler = sampler
        self.pos = pos
        self.previous_voxels = None
        
        self.xstep = sampler.xs[1] - sampler.xs[0]
        self.ystep = sampler.ys[1] - sampler.ys[0]
        self.zstep = sampler.zs[1] - sampler.zs[0]
        
        self.voxels = {}
        
        for p in pos:
            position = [p[0] - (self.xstep / 2), p[1] - (self.ystep / 2), p[2] - (self.zstep / 2)]
            i, j, k = np.where(np.isclose(sampler.xs, p[0]))[0][0], np.where(np.isclose(sampler.ys, p[1]))[0][0], np.where(np.isclose(sampler.zs, p[2]))[0][0]
            voxel : Poly3DCollection = plotCubeAt(position, (self.xstep, self.ystep, self.zstep), facecolors='C4', edgecolors='C2', shade=True)
            self.voxels[(i, j, k)] = voxel
            ax.add_collection3d(voxel)
            
        self.label_z = ax.text2D(
            0.95, 0.95, '', style='italic', 
            bbox={'facecolor': 'purple', 'alpha': 0.5, 'pad': 10},
            visible=False
        )


    def handle_keypress(self, event) -> None:
        if event.key == 'right':
            self.index = (self.index + 1) if self.index < len(self.sampler.zs) - 1 else 0
        elif event.key == 'left':
            self.index = (self.index - 1) if self.index > 0 else len(self.sampler.zs) - 1
        elif event.key == 'escape':
            self.index = -1
        self.update()
        
    
    def update(self) -> None:
        if self.index < 0:
            for voxel in self.voxels.values():
                voxel.set_visible(True)
                self.label_z.set_visible(False)
        else:
            self.label_z.set_text(f'Current z : {self.sampler.zs[self.index]}')
            self.label_z.set_visible(True)
            for coords, voxel in self.voxels.items():
                voxel.set_visible(coords[2] == self.index)
        
        self.fig.canvas.draw()

In [58]:
visible = True
def handle_key(event):
    global visible, scatter, rr100
    if event.key == 'x':
        visible = not visible
        scatter.set_visible(visible)
        rr100.set_visible(visible)

fig = plt.figure()
fig.canvas.mpl_connect('key_press_event', handle_key)
ax = fig.add_subplot(projection='3d')
t = VoxelPlotter(fig, ax, sampler, pos)
scatter = ax.scatter(nxs, nys, nzs, c='red', marker='x', label='Unreachable')

ax.set_xlim(-1.25, 0.5)

rr100: Poly3DCollection = plotCubeAt([-0.45981, -0.65850 / 2, 0.0], size=(0.45981 * 2, 0.65850, 0.80091), edgecolor='C2', facecolors=np.repeat('C0', 6), linewidths=1, alpha=0.5)
rr100.set_label('RR100')
ax.add_collection3d(rr100)

ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()

plt.show()

['_PROPERTIES_EXCLUDED_FROM_SET', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_agg_filter', '_alias_map', '_alpha', '_animated', '_axes', '_bbox_patch', '_callbacks', '_char_index_at', '_charsize_cache', '_clipon', '_clippath', '_cm_set', '_color', '_default_contains', '_fontproperties', '_fully_clipped_to_axes', '_get_dist_to_box', '_get_layout', '_get_multialignment', '_get_rendered_text_width', '_get_wrap_line_width', '_get_wrapped_text', '_get_xy_display', '_gid', '_horizontalalignment', '_in_layout', '_internal_update', '_label', '_linespacing', '_mouseover', '_multialignment', '_parse_math', '_path_effects', '_picker', '_preprocess_math', '_rasterized', '_remove_method

In [ ]:
manipulable, n_manipulable = sampler.sample_manipulable_points(reached)

In [ ]:
manipulable = np.array(manipulable)
n_manipulable = np.array(n_manipulable)
xs, ys, zs = manipulable[:, 0, 3], manipulable[:, 1, 3], manipulable[:, 2, 3]
nxs, nys, nzs = n_manipulable[:, 0, 3], n_manipulable[:, 1, 3], n_manipulable[:, 2, 3]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(xs, ys, zs, c='blue', marker='o', label='Reached AND manipulable')
ax.scatter(nxs, nys, nzs, c='red', marker='x', label='Not manipulable or induces collision')

ax.set_xlim(-1.0, 0.5)

rr100: Poly3DCollection = plotCubeAt([-0.45981, -0.65850 / 2, 0.0], size=(0.45981 * 2, 0.65850, 0.80091), edgecolors='C4', facecolors=np.repeat('C0', 6), linewidths=1, alpha=0.5)
rr100.set_label('RR100')
ax.add_collection3d(rr100)

ax.set_aspect('equal')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()

plt.show()